In [1]:
!pip install /kaggle/input/dicomsdl-offline-installer/dicomsdl-0.109.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
!pip install --no-deps /kaggle/input/keras-cv-attention-models/keras_cv_attention_models-1.3.9-py3-none-any.whl

Processing /kaggle/input/dicomsdl-offline-installer/dicomsdl-0.109.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
Processing /kaggle/input/keras-cv-attention-models/keras_cv_attention_models-1.3.9-py3-none-any.whl


In [2]:
import os
import time
import numpy as np
import pandas as pd
import cv2
import dicomsdl
from tqdm import tqdm
import math

from enum import Enum
import matplotlib.pyplot as plt

import tensorflow as tf
from joblib import Parallel, delayed

# Creating and preproccesing DataSet
1. from traing set get 'site_id','patient_id','image_id', 'laterality','view','age','implant','cancer'
2. Since routine examination includes MLO and CC projections, we droped all another projections
3. Croping and fliping image ro one side
4. Merge MLO and CC layers, cause we need a general picture of the study

In [3]:
csv_path = '/kaggle/input/rsna-breast-cancer-detection/train.csv'

imgs_path = '/kaggle/input/rsna-breast-cancer-detection/train_images'
imgs_path_png = '/kaggle/input/rsna-mammography-images-as-pngs/images_as_pngs_cv2_vl_asp_1024/train_images_processed_cv2_vl_asp_1024'
test_csv_path = '/kaggle/input/rsna-breast-cancer-detection/test.csv'
test_imgs_path = '/kaggle/input/rsna-breast-cancer-detection/test_images'

In [4]:
try:
    TPU = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', TPU.master())
except ValueError:
    print('Running on CPU or GPU')
    TPU = None
    
if TPU:
    IS_TPU = True
    tf.config.experimental_connect_to_cluster(TPU)
    tf.tpu.experimental.initialize_tpu_system(TPU)
    STRATEGY = tf.distribute.experimental.TPUStrategy(TPU)
else:
    IS_TPU = False
    STRATEGY = tf.distribute.MirroredStrategy()


N_REPLICAS = STRATEGY.num_replicas_in_sync
BATCH_SIZE = 8 * N_REPLICAS
N_EPOCHS = 10

#IMAGE_SHAPE
#img_shape = (256,448)
img_shape = (768,1344)


TRAIN_SETS=10


print(f'N_REPLICAS: {N_REPLICAS}, IS_TPU: {IS_TPU}')

policy = tf.keras.mixed_precision.Policy('float32')
tf.keras.mixed_precision.set_global_policy(policy)

Running on CPU or GPU
N_REPLICAS: 2, IS_TPU: False


In [5]:
from enum import Enum
class Side(Enum):
    LEFT = 1
    RIGHT = 2

PreprocessImage base class

In [6]:
def crop(img):
    # Binarize the image
    bin_pixels = cv2.threshold(img, 20, 255, cv2.THRESH_BINARY)[1]
   
    # Make contours around the binarized image, keep only the largest contour
    contours, _ = cv2.findContours(bin_pixels, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    contour = max(contours, key=cv2.contourArea)

    # Create a mask from the largest contour
    mask = np.zeros(img.shape, np.uint8)
    cv2.drawContours(mask, [contour], -1, 255, cv2.FILLED)
   
    # Use bitwise_and to get masked part of the original image
    out = cv2.bitwise_and(img, mask)
    
    # get bounding box of contour
    y1, y2 = np.min(contour[:, :, 1]), np.max(contour[:, :, 1])
    x1, x2 = np.min(contour[:, :, 0]), np.max(contour[:, :, 0])
    
    x1 = int(0.99 * x1)
    x2 = int(1.01 * x2)
    y1 = int(0.99 * y1)
    y2 = int(1.01 * y2)


    return out[y1:y2, x1:x2]

In [7]:
class PreprocessImage:
    
    def __init__(self, img_path, img_size=(512,512),flip=True, side=Side.LEFT, crop=True):
        self.img_path = img_path
        self.img_size = img_size
        self.flip = flip
        self.side = side
        self.crop = crop
    
    def loadXray(self, file_path):
        pass
    
    def getExt(self):
        pass
    
    def getImgPath(self):
        return self.img_path
    
    def readXray(self, file_path):
        img = self.loadXray(file_path)
        
        if self.flip:
            if self.determine_side(img) != self.side:
                img = cv2.flip(img,1)

        if self.crop:
            """
            coords = cv2.findNonZero(img) # Find all non-zero points (text)
            x, y, w, h = cv2.boundingRect(coords) # Find minimum spanning bounding box
            img = img[y:y+h, x:x+w]
            """
            img = crop(img)

        if self.img_size:
            img = cv2.resize(img, self.img_size, interpolation=cv2.INTER_AREA)

        # Add channel dim at First
        img = img[:,:,np.newaxis]

        # Converting img to float32
        img = img / np.max(img)
        img = img.astype("float32")

        return img
    
    def determine_side(self,img, threshold = 5):

        w0 = img.shape[1]
        #where score was 0.5
        
        if img[:,int(-w0 * 0.10):].sum() > img[:,:int(w0 * 0.10)].sum():
            return Side.RIGHT
        else:
            return Side.LEFT
        
        """width = img.shape[1]
        if sum(sum(img[:, :1])) > sum(sum(img[:, width-1:])): 
            return Side.LEFT
        else:
            return Side.RIGHT"""
    
    
    def getSplitXray(self, mlo_path,cc_path,age=0):
    
        mlo = self.readXray(self.img_path+mlo_path)
        cc  = self.readXray(self.img_path+cc_path)
        zeros = np.full(( self.img_size[1],self.img_size[0],1 ), min(age/100,1.0), dtype='float32')

        img = np.concatenate((mlo, cc,zeros), axis=2) 
        return img

**Loader for DICOMS**

In [8]:
class PreprocessImageDICOM(PreprocessImage):

    def loadXray(self, file_path):
        dicom = dicomsdl.open(file_path)
        data = dicom.pixelData(storedvalue=False)  # storedvalue = True for int16 return otherwise float32
        
        try:
            data = self.voi_lut(data,dicom)
        except:
            pass
        

        if dicom.PhotometricInterpretation == "MONOCHROME1":
            data = np.max(data) - data
            
        data = (data - data.min()) / (data.max() - data.min())

        # Convert to uint8 image in range [0, 255]
        data = (data * 255).astype(np.uint8)

        return data
    
    def voi_lut(self,img,dataset):
         # Load only the variables we need
        center = dataset["WindowCenter"]
        width = dataset["WindowWidth"]
        bits_stored = dataset["BitsStored"]
        voi_lut_function = dataset["VOILUTFunction"]

        # For sigmoid it's a list, otherwise a single value
        if isinstance(center, list):
            center = center[0]
        if isinstance(width, list):
            width = width[0]

        # Set y_min, max & range
        y_min = 0
        y_max = float(2**bits_stored - 1)
        y_range = y_max

        # Function with default LINEAR (so for Nan, it will use linear)
        if voi_lut_function == "SIGMOID":
            img = y_range / (1 + np.exp(-4 * (img - center) / width)) + y_min
        else:
            # Checks width for < 1 (in our case not necessary, always >= 750)
            center -= 0.5
            width -= 1

            below = img <= (center - width / 2)
            above = img > (center + width / 2)
            between = np.logical_and(~below, ~above)

            img[below] = y_min
            img[above] = y_max
            if between.any():
                img[between] = (
                    ((img[between] - center) / width + 0.5) * y_range + y_min
                )
                
        return img
    
    def getExt(self):
        return "dcm"

**Loader for pngs**

In [9]:
class PreprocessImagePng(PreprocessImage):
    def loadXray(self, file_path):
        img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
        return img
    
    def getExt(self):
        return "png"


**DataGenerator for Keras model**

In [10]:
from tensorflow.keras.utils import Sequence

class XrayDataGenerator (Sequence):
    
    def __init__(self, loader: PreprocessImage, batch_size=4, new_df=None, saveImages=False, 
                 saveDF:str=None, shuffle=True,isTest=False, trainSet=0, validation=False, valSplit = 0):
        self.loader = loader
        self.saveImages=saveImages
        self.saveDF = saveDF
        if new_df is not None:
            if isinstance(new_df, str):
                self.new_df = pd.read_csv(new_df)
            if isinstance(new_df, pd.DataFrame):
                self.new_df = new_df

        #colums of new dataSet
        #cancer column in train is 1 or 0
        #cancer column in test set is [patient_id + _ + laterality]
        self.columns_names=["patient_id","pred_name","image","age","implant",'cancer']
        #this generator for train or testing
        self.isTest = isTest
        
        #trainSet is part of full train set depended of cancer positive data
        #len of newSet = len(cancer_positive_data) + len(cancer_positive_data)*trainSet
        self.trainSet = trainSet
        self.valSplit = valSplit
        self.validation = validation
        #if its generator for test set trainSet to 0
        if self.isTest:
            self.trainSet=0
            self.valSplit=0
            self.validation = False
        
        if new_df is None:
            self.new_df = pd.DataFrame(columns = self.columns_names)
            
        self.n = len(self.new_df)
            
        self.batch_size = batch_size

        self.shuffle = shuffle
        self.random_state = 43
        self.full_df = pd.DataFrame()
    
    def generateDF(self, csv_path:str):
        df = pd.read_csv(csv_path)
        cols = ['site_id','patient_id','image_id','laterality','view','age','implant']
        if self.isTest!=True:
            cols.append('cancer')

        df = df[cols]
        df['age'] = df['age'].fillna(0)
        df['implant'] = df['implant'].fillna(0)

        self.new_df = pd.DataFrame(columns = self.columns_names)

        self.new_df[['patient_id','age','implant']] = df[['patient_id','age','implant']]
        if not self.isTest:
            self.new_df['cancer'] = df['cancer']
            
        self.new_df['pred_name'] = df['patient_id'].astype(str)+'_'+df["laterality"]
        self.new_df['image'] = self.loader.getImgPath()+'/'+df['patient_id'].astype(str)+'/'+df['image_id'].astype(str)+'.'+self.loader.getExt()
        
        if self.saveDF is not None:
            self.new_df.to_csv(self.saveDF,index=False)
        
        self.new_df = self.cropSamples()
        
        return self.new_df
    
    def cropSamples(self):
          
        if self.trainSet!=0:
            c_df = self.new_df[self.new_df['cancer']==1]
            n_df = self.new_df[self.new_df['cancer']==0].sample(frac=1,random_state=self.random_state).reset_index(drop=True).head( len(c_df)*self.trainSet)
            self.new_df = c_df.append(n_df).sample(frac=1,random_state=self.random_state).reset_index(drop=True)
     
             
        indFirst = 0
        indLast = len(self.new_df)
        
        if self.valSplit:
            
            self.full_df = self.new_df
            
            if self.validation:
                indFirst = int(len(self.new_df)*(1-self.valSplit))
            else:
                indLast = int(len(self.new_df)*(1-self.valSplit))
        
        self.new_df = self.new_df[indFirst:indLast]
        
        self.n = len(self.new_df)
        
        return self.new_df
    
    def getValidationSet(self):
        if self.valSplit:
            genVal = XrayDataGenerator(imgLoader,new_df=self.full_df,trainSet=self.trainSet,valSplit=self.valSplit,validation=True)
            genVal.cropSamples()
            return genVal
        else:
            raise Exception('valSplit is zero')
    
    
    def getImage(self, index):
        
        img = self.loader.readXray(self.new_df.loc[index]["image"])  
        return img
    
    
    def on_epoch_end(self):
        if self.shuffle:
            self.new_df = self.new_df.sample(frac=1).reset_index(drop=True)
    
    
    def __getitem__(self, index):
        
        last_ind = (index + 1) * self.batch_size
        if last_ind > self.n:
            last_ind = self.n
        
        batches = self.new_df[index * self.batch_size:last_ind]

        X_batch_1 = np.array(
                    Parallel(n_jobs=4)(
                    delayed(self.getImage)(index)
                    for index  in batches.index.values    
                    ) )
                
        #X_batch_1 = (np.asarray([self.getImage(index) for index, x in batches.iterrows()])).astype('float32')
        X_batch_2 =  batches[['age','implant']].values.astype('float32')
        #X_batch_2 = batches[['age']].values.astype('float32')
        #X_batch_2 = np.minimum( (batches[['age']].values /100.0), 1.0).astype('float32')

        #return [X_batch_1,X_batch_2],y_batch
        if self.isTest:
            return [X_batch_1,X_batch_2]
            #return X_batch_1
        else:
            y_batch = batches["cancer"].values.astype('float32')
            return [X_batch_1,X_batch_2],y_batch
            #return X_batch_1,y_batch
    
    def __len__(self):
        if self.n % self.batch_size:
            return (self.n // self.batch_size)+1
        else:
            return (self.n // self.batch_size)
       

**Test set Generator**

In [11]:
dicomLoader = PreprocessImageDICOM(test_imgs_path,img_shape,flip=False)
testGen = XrayDataGenerator(dicomLoader,batch_size=BATCH_SIZE, isTest=True)
test_df = testGen.generateDF(test_csv_path)

# **KERAS simple model**

In [12]:
import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, BatchNormalization, Input
from tensorflow.keras.layers import AveragePooling2D, GlobalAveragePooling2D, AveragePooling3D
from tensorflow.keras.layers import Add, LayerNormalization, Concatenate, Dropout, Average
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from keras_cv_attention_models import convnext,resnet_family

In [13]:
class pFBeta(tf.keras.metrics.Metric):
    def __init__(self, beta=1, name='pF1', **kwargs):
        super().__init__(name=name, **kwargs)
        self.beta = beta
        self.epsilon = 1e-10
        self.pos = self.add_weight(name='pos', initializer='zeros')
        self.ctp = self.add_weight(name='ctp', initializer='zeros')
        self.cfp = self.add_weight(name='cfp', initializer='zeros')
        
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.clip_by_value(y_pred, 0, 1)
        pos = tf.cast(tf.reduce_sum(y_true), tf.float32)
        ctp = tf.cast(tf.reduce_sum(y_pred[y_true == 1]), tf.float32)
        cfp = tf.cast(tf.reduce_sum(y_pred[y_true == 0]), tf.float32)
        self.pos.assign_add(pos)
        self.ctp.assign_add(ctp)
        self.cfp.assign_add(cfp)
        
    def result(self):
        beta2 = self.beta * self.beta
        prec = self.ctp / (self.ctp + self.cfp + self.epsilon)
        reca = self.ctp / (self.pos + self.epsilon)
        return (1 + beta2) * prec * reca / (beta2 * prec + reca)
        
    def reset_state(self):
        self.pos.assign(0.)
        self.ctp.assign(0.)
        self.cfp.assign(0.)

In [14]:
def getModel():
    
    input_image = Input(shape=(img_shape[1],img_shape[0],1))
    
    prepare_image = tf.repeat(input_image, repeats=3, axis=3)
       
    cnn = convnext.ConvNeXtV2Tiny(input_shape=(img_shape[1], img_shape[0],3), pretrained=None, num_classes=0)(prepare_image)
    #cnn = resnet_family.ResNeXt101(input_shape=(img_shape[1], img_shape[0],3), pretrained=None, num_classes=0)(prepare_image)
    final_layer = GlobalAveragePooling2D()(cnn)
    
    age = Input(shape=(2,),name='age_input') #with implants
    
    final_layer = Concatenate()([final_layer,age])
    
      #ff
    _final_layer = Dense(final_layer.shape[-1]*4,activation='elu')(final_layer)
    _final_layer = Dense(final_layer.shape[-1])(_final_layer)
    
    #relu-add-norm
    _final_layer = keras.activations.relu(_final_layer)
    final_layer = Add()([_final_layer,final_layer])
    final_layer = LayerNormalization(epsilon=1e-6)(final_layer)
    
        
    final_layer = Dense(1,activation='sigmoid')(final_layer)

    model = Model([input_image,age], final_layer)
    
    model.load_weights('/kaggle/input/n-train-one-img-weights-tpu/n_model_768_1344_convnext_ff_ss38_noflip_ep12.h5')
    
    model.trainable = False

    model.compile()

    
    return model


In [15]:
with STRATEGY.scope():

    model = getModel()
    
    model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1344, 768, 1 0                                            
__________________________________________________________________________________________________
tf.repeat (TFOpLambda)          (None, 1344, 768, No 0           input_1[0][0]                    
__________________________________________________________________________________________________
convnext_v2_tiny (Functional)   (None, 42, 24, 768)  27864960    tf.repeat[0][0]                  
__________________________________________________________________________________________________
global_average_pooling2d (Globa (None, 768)          0           convnext_v2_tiny[0][0]           
______________________________________________________________________________________________

In [16]:
predict = model.predict(testGen)

In [17]:
submission = {'prediction_id': test_df['pred_name'].values ,
              "cancer":predict[:,0]}
submission = pd.DataFrame(submission)
submission = pd.DataFrame(submission.groupby('prediction_id')['cancer'].mean()).reset_index()
submission['cancer'] = np.int8(submission['cancer'].values > 0.5)
submission.to_csv("submission.csv", index=False)
submission

,prediction_id,cancer
0,10008_L,0
1,10008_R,0
